In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
from datetime import datetime
import csv
from keras_facenet import FaceNet
from joblib import load

In [1]:
!pip install joblib
!pip install matplotlib
!pip install mtcnn

In [2]:
import urllib.request

# Disable SSL certificate verification (not recommended for production)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Then proceed with your code


In [4]:
# face recognition part II
#IMPORT
import csv
from datetime import datetime
import cv2 as cv
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet
from joblib import load
#INITIALIZE
facenet = FaceNet()
faces_embeddings = np.load("/Users/pradeepslayer/Downloads/project k/face_moham_viluvalu.npz ")
Y = faces_embeddings['arr_1']
encoder = LabelEncoder()
encoder.fit(Y)
haarcascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
model = load(open("na_svm_model.joblib", 'rb'))

URLError: <urlopen error [Errno 60] Operation timed out>

In [ ]:
import os
import csv
import cv2 as cv
import numpy as np
from datetime import datetime

from werkzeug.utils import secure_filename
from flask import Flask, render_template, request

app = Flask(__name__)

UPLOAD_FOLDER = 'C:\\Users\\new\\Desktop\\ProjectK\\uploads'

def take_attendance(frame):
    try:
        # Convert frame to numpy array if it's not already
        if not isinstance(frame, np.ndarray):
            frame = cv.imread(frame)
        
        rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = haarcascade.detectMultiScale(gray_img, 1.3, 7)
        for x,y,w,h in faces:
            img = rgb_img[y:y+h, x:x+w]
            img = cv.resize(img, (160,160)) #1x160x160x3
            img = np.expand_dims(img,axis=0)
            ypred = facenet.embeddings(img)
            face_name = model.predict(ypred)
            final_name = encoder.inverse_transform(face_name)[0]
            print(final_name)

        # Get current date
        current_date = datetime.now().strftime('%Y-%m-%d')
        
        # CSV file path for the current date
        #csv_file_path = f'C:\\Users\\new\\Desktop\\ProjectK\\attendance_logs\\attendance_{current_date}.csv'
        # CSV file path for the current date
        csv_directory = 'C:\\Users\\new\\Desktop\\ProjectK\\attendance_logs'
        if not os.path.exists(csv_directory):
            os.makedirs(csv_directory)
        csv_file_path = os.path.join(csv_directory, f'attendance_{current_date}.csv')


        existing_names = []
        if os.path.exists(csv_file_path):
            with open(csv_file_path, mode='r', newline='') as file:
                reader = csv.reader(file)
                for row in reader:
                    existing_names.append(row[0])

        # Open CSV file in append mode
        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)

            # Iterate through detected faces
            for x, y, w, h in faces:
                img = rgb_img[y:y+h, x:x+w]
                img = cv.resize(img, (160,160)) # Resize image
                img = np.expand_dims(img, axis=0)
                ypred = facenet.embeddings(img)
                face_name = model.predict(ypred)
                final_name = encoder.inverse_transform(face_name)[0]
                print(final_name)

                # Get current time
                current_time = datetime.now()
                # Format time
                formatted_time = current_time.strftime('%H:%M:%S')

                # Write name, time to CSV
                if final_name not in existing_names:
                    writer.writerow([final_name, formatted_time])
                    existing_names.append(final_name)
    except Exception as e:
        print(f"Error in take_attendance function: {e}")

def generate_unique_filename(filename):
    filename_without_extension, extension = os.path.splitext(filename)
    return f"{filename_without_extension}_{os.urandom(8).hex()}{extension}"

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload_image', methods=['POST'])
def upload_image():
    if request.method == 'POST':
        # Ensure the directory exists
        os.makedirs(UPLOAD_FOLDER, exist_ok=True)

        image = request.files['image']
        if image:
            filename = secure_filename(image.filename)
            filename = generate_unique_filename(filename)
            image_path = os.path.join(UPLOAD_FOLDER, filename)
            image.save(image_path)
            take_attendance(image_path)
            return 'Attendance Taken Successfully'

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
